In [4]:
import json
import os
from typing import List, Any, Tuple, Dict, Optional
from langchain_openai import ChatOpenAI
from openai import OpenAI
from dataclasses import dataclass

In [2]:
@dataclass
class RAGEvaluationResult:
    """Structure to hold evaluation results for a single question"""
    question: str
    context: str
    generated_answer: str
    relevance_score: float
    correctness_score: float
    faithfulness_score: float
    hallucination_score: float
    detailed_feedback: Dict[str, str]
    overall_score: float

In [5]:
@dataclass
class OpenRouterModel:
    api_key: Optional[str] = os.getenv("OPENROUTER_API_KEY")

    def __post_init__(self):
        if not self.api_key:
            raise ValueError("Please set a valid api key in the environment variables")
        self.client = OpenAI(
            base_url="https://openrouter.ai/api/v1",
            api_key=self.api_key,
        )
        
        self.model = "meta-llama/llama-3.1-70b-instruct"

    def invoke(self, prompt: str, temperature: float = 0.0, max_tokens: int = 2000) -> str:
        """Generate response using Llama 3.1 70B Instruct"""
        try:
            response = self.client.chat.completions.create(
                model=self.model,
                messages=[{"role": "user", "content": prompt}],
                temperature=temperature,
                max_tokens=max_tokens
            )
            return response.choices[0].message.content
        except Exception as e:
            print(f"Error calling OpenRouter API: {e}")
            return ""
        

In [8]:
llama_model = OpenRouterModel()
prompt = "What is the capital of Spain?"
response = llama_model.invoke(prompt)
print("Response:", response)

Response: The capital of Spain is Madrid.
